In [1]:
from PIL import Image
import os

# === Config ===
image_path = "input_data/X_test/"  # Đường dẫn đến ảnh gốc
ROWS = 3
COLS = 5

def split_image(image_path, rows, cols):
    """Đọc ảnh và cắt thành rows x cols, trả về danh sách các mảnh."""
    img = Image.open(image_path)
    width, height = img.size

    piece_width = width // cols
    piece_height = height // rows

    pieces = []
    for r in range(rows):
        for c in range(cols):
            left = c * piece_width
            upper = r * piece_height
            right = left + piece_width
            lower = upper + piece_height
            piece = img.crop((left, upper, right, lower))
            pieces.append(piece)

    return pieces

def load_all_pieces(image_dir, rows, cols):
    """
    Duyệt toàn bộ ảnh trong thư mục và trả về mảng 3 chiều:
    all_pieces[image_index][row][col] = PIL.Image
    """
    all_pieces = []
    image_files = sorted(
        [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    )

    print(f"Found {len(image_files)} images in {image_dir}")

    for filename in image_files:
        image_path = os.path.join(image_dir, filename)
        print(f"- Processing {filename}...")
        pieces = split_image(image_path, rows, cols)
        all_pieces.append(pieces)

    print(f"\nTotal images processed: {len(all_pieces)}")
    return all_pieces

all_pieces = load_all_pieces(image_path, ROWS, COLS)
# if DEBUG:
#     all_pieces = all_pieces[:20]  # Giới hạn để debug nhanh

# Ví dụ: xem thông tin
print(f"Total images: {len(all_pieces)}")  # số lượng ảnh
if all_pieces:
    print(f"Rows per image: {len(all_pieces[0])}")


Found 100 images in input_data/X_test/
- Processing Alfred_Sisley_177_shuffled.jpg...
- Processing Alfred_Sisley_187_shuffled.jpg...
- Processing Alfred_Sisley_69_shuffled.jpg...
- Processing Amedeo_Modigliani_37_shuffled.jpg...
- Processing Andrei_Rublev_25_shuffled.jpg...
- Processing Andrei_Rublev_37_shuffled.jpg...
- Processing Andy_Warhol_119_shuffled.jpg...
- Processing Caravaggio_54_shuffled.jpg...
- Processing Caravaggio_7_shuffled.jpg...
- Processing Diego_Rivera_19_shuffled.jpg...
- Processing Edgar_Degas_108_shuffled.jpg...
- Processing Edgar_Degas_184_shuffled.jpg...
- Processing Edgar_Degas_189_shuffled.jpg...
- Processing Edgar_Degas_278_shuffled.jpg...
- Processing Edgar_Degas_299_shuffled.jpg...
- Processing Edgar_Degas_331_shuffled.jpg...
- Processing Edgar_Degas_340_shuffled.jpg...
- Processing Edgar_Degas_345_shuffled.jpg...
- Processing Edgar_Degas_462_shuffled.jpg...
- Processing Edgar_Degas_486_shuffled.jpg...
- Processing Edgar_Degas_502_shuffled.jpg...
- Process

In [ ]:
import numpy as np
from PIL import Image
import os
import csv
import random
from tqdm import tqdm
import pandas as pd

# =========================
# COST: MSE theo viền kề
# =========================
def _to_np_rgb(pil_img):
    return np.asarray(pil_img.convert("RGB"), dtype=np.float32)

def _border_arrays(pieces):
    arrs = [_to_np_rgb(p) for p in pieces]
    tops    = [a[0, :, :]  for a in arrs]  # (W,3)
    bottoms = [a[-1, :, :] for a in arrs]  # (W,3)
    lefts   = [a[:, 0, :]  for a in arrs]  # (H,3)
    rights  = [a[:, -1, :] for a in arrs]  # (H,3)
    return tops, bottoms, lefts, rights

def _mse_rgb_edge(edge1, edge2):
    L = min(edge1.shape[0], edge2.shape[0])
    if L <= 0:
        return 0.0
    diff = edge1[:L].astype(np.float32) - edge2[:L].astype(np.float32)
    return float(np.mean(diff * diff))

def compute_cost_matrix_mse(pieces):
    """
    - H[i,j]: cost đặt j bên phải i  -> so sánh right(i) vs left(j)
    - V[i,j]: cost đặt j bên dưới i -> so sánh bottom(i) vs top(j)
    """
    n = len(pieces)
    H = np.zeros((n, n), dtype=np.float64)
    V = np.zeros((n, n), dtype=np.float64)

    tops, bottoms, lefts, rights = _border_arrays(pieces)
    for i in range(n):
        r_i = rights[i]
        b_i = bottoms[i]
        for j in range(n):
            if i == j:
                continue
            H[i, j] = _mse_rgb_edge(r_i, lefts[j])
            V[i, j] = _mse_rgb_edge(b_i, tops[j])
    return H, V


# =========================
# LOCAL SEARCH (exhaustive)
# Chromosome = c->o (cell -> piece)
# =========================
class LocalSearchPuzzleSolver:
    def __init__(self, rows, cols, max_iters=1000, restarts=5, rng_seed=None):
        self.rows = rows
        self.cols = cols
        self.max_iters = max_iters
        self.restarts = restarts
        self.rng = random.Random(rng_seed)

    # ------ utils ------
    def _repair_perm(self, chrom):
        """Đảm bảo chrom là hoán vị 0..N-1 (loại trùng/ngoài miền, chèn thiếu)."""
        n = self.rows * self.cols
        used, out = set(), []
        for g in chrom:
            if isinstance(g, (int, np.integer)) and 0 <= g < n and g not in used:
                out.append(int(g)); used.add(int(g))
        out.extend([x for x in range(n) if x not in used])
        return out[:n]

    def _fitness(self, chrom, H, V):
        total = 0.0
        grid = np.array(chrom, dtype=int).reshape(self.rows, self.cols)
        for r in range(self.rows):
            for c in range(self.cols):
                cur = grid[r, c]
                if c < self.cols - 1:
                    total += H[cur, grid[r, c+1]]
                if r < self.rows - 1:
                    total += V[cur, grid[r+1, c]]
        return float(total)

    def _random_chrom(self, n):
        base = list(range(n))
        self.rng.shuffle(base)
        return base

    # ------ láng giềng: SWAP & BLOCK-SWAP (duyệt hết) ------
    def _best_improvement_swap_and_blockswap(self, chrom, H, V, base_fit):
        R, C = self.rows, self.cols
        best_neighbor, best_delta = None, 0.0

        # 1) Tất cả SWAP i<j
        n = len(chrom)
        for i in range(n - 1):
            for j in range(i + 1, n):
                cand = chrom[:]
                cand[i], cand[j] = cand[j], cand[i]
                cand = self._repair_perm(cand)
                d = self._fitness(cand, H, V) - base_fit
                if d < best_delta:
                    best_delta, best_neighbor = d, cand

        # 2) Tất cả BLOCK-SWAP (mọi h,w; mọi cặp vị trí; cho phép overlap)
        grid0 = np.array(chrom, dtype=int).reshape(R, C)
        for h in range(1, R + 1):
            for w in range(1, C + 1):
                positions = [(r, c) for r in range(0, R - h + 1)
                                   for c in range(0, C - w + 1)]
                m = len(positions)
                if m <= 1:
                    continue
                for a in range(m - 1):
                    r0, c0 = positions[a]
                    for b in range(a + 1, m):
                        r1, c1 = positions[b]
                        if r0 == r1 and c0 == c1:
                            continue
                        g = grid0.copy()
                        src = g[r0:r0+h, c0:c0+w].copy()
                        dst = g[r1:r1+h, c1:c1+w].copy()
                        g[r0:r0+h, c0:c0+w] = dst
                        g[r1:r1+h, c1:c1+w] = src
                        cand = self._repair_perm(g.reshape(-1).tolist())
                        d = self._fitness(cand, H, V) - base_fit
                        if d < best_delta:
                            best_delta, best_neighbor = d, cand

        return best_neighbor, best_delta

    # ------ láng giềng: “1-đổi-2” dọc & ngang (duyệt hết) ------
    def _best_improvement_one_with_two(self, chrom, H, V, base_fit):
        """
        Dọc:  chọn dải cột (w, c0), phủ toàn bộ R; cắt theo chiều dọc tại r1<r2:
              [Top=A, Mid=B, Bot=C] -> [C, A, B]
        Ngang: chọn dải hàng (h, r0), phủ toàn bộ C; cắt theo chiều ngang tại c1<c2:
              [Left=A, Mid=B, Right=C] -> [C, A, B]
        """
        R, C = self.rows, self.cols
        grid0 = np.array(chrom, dtype=int).reshape(R, C)
        best_neighbor, best_delta = None, 0.0

        # --- 1) Dọc: chọn w và c0; cắt r1<r2 ---
        # r1, r2 là đường cắt giữa các hàng => trong [1..R-1], chọn 2 => C(R-1, 2)
        if R >= 3:
            cut_rows = [i for i in range(1, R)]  # vị trí cắt hợp lệ
            # duyệt bề rộng w = 1..C, và vị trí bắt đầu c0
            for w in range(1, C + 1):
                for c0 in range(0, C - w + 1):
                    for a in range(len(cut_rows) - 1):
                        r1 = cut_rows[a]
                        for b in range(a + 1, len(cut_rows)):
                            r2 = cut_rows[b]
                            # ba dải theo chiều dọc trong (tất cả R) nhưng chỉ tác động trên cột [c0..c0+w-1]
                            g = grid0.copy()
                            A = g[0:r1,        c0:c0+w].copy()   # top
                            B = g[r1:r2,       c0:c0+w].copy()   # mid
                            Cbot = g[r2:R,     c0:c0+w].copy()   # bottom
                            # xếp lại: [C, A, B]
                            # đặt C lên trên
                            g[0:len(Cbot),     c0:c0+w] = Cbot
                            # tiếp A
                            g[len(Cbot):len(Cbot)+len(A), c0:c0+w] = A
                            # cuối B
                            g[len(Cbot)+len(A):len(Cbot)+len(A)+len(B), c0:c0+w] = B
                            cand = self._repair_perm(g.reshape(-1).tolist())
                            d = self._fitness(cand, H, V) - base_fit
                            if d < best_delta:
                                best_delta, best_neighbor = d, cand

        # --- 2) Ngang: chọn h và r0; cắt c1<c2 ---
        # c1, c2 là đường cắt giữa các cột => trong [1..C-1], chọn 2 => C(C-1, 2)
        if C >= 3:
            cut_cols = [j for j in range(1, C)]
            for h in range(1, R + 1):
                for r0 in range(0, R - h + 1):
                    for a in range(len(cut_cols) - 1):
                        c1 = cut_cols[a]
                        for b in range(a + 1, len(cut_cols)):
                            c2 = cut_cols[b]
                            g = grid0.copy()
                            A = g[r0:r0+h, 0:c1       ].copy()   # left
                            B = g[r0:r0+h, c1:c2      ].copy()   # mid
                            Cright = g[r0:r0+h, c2:C  ].copy()   # right
                            # xếp lại: [C, A, B] trong vùng hàng [r0..r0+h-1], phủ toàn bộ chiều ngang
                            wC = Cright.shape[1]; wA = A.shape[1]; wB = B.shape[1]
                            # đặt C sang trái
                            g[r0:r0+h, 0:wC] = Cright
                            # rồi A
                            g[r0:r0+h, wC:wC+wA] = A
                            # rồi B
                            g[r0:r0+h, wC+wA:wC+wA+wB] = B
                            cand = self._repair_perm(g.reshape(-1).tolist())
                            d = self._fitness(cand, H, V) - base_fit
                            if d < best_delta:
                                best_delta, best_neighbor = d, cand

        return best_neighbor, best_delta

    # ------ bước best-improvement: gộp cả 3 nhóm láng giềng ------
    def _best_improvement_step(self, chrom, H, V):
        base_fit = self._fitness(chrom, H, V)

        # 1) swap + block-swap
        nb1, d1 = self._best_improvement_swap_and_blockswap(chrom, H, V, base_fit)

        # 2) one-with-two (dọc + ngang)
        nb2, d2 = self._best_improvement_one_with_two(chrom, H, V, base_fit)

        # chọn cải thiện tốt nhất
        if nb1 is None and nb2 is None:
            return None, 0.0
        if nb1 is None or (nb2 is not None and d2 < d1):
            return nb2, d2
        return nb1, d1

    # ------ main ------
    def run(self, pieces):
        """Trả về (best_order, best_fitness). Chromosome là c->o."""
        n = self.rows * self.cols
        H, V = compute_cost_matrix_mse(pieces)

        global_best, global_best_fit = None, float("inf")
        for _ in range(self.restarts):
            cur = self._repair_perm(self._random_chrom(n))
            cur_fit = self._fitness(cur, H, V)

            for _ in range(self.max_iters):
                nb, delta = self._best_improvement_step(cur, H, V)
                if nb is None or delta >= 0.0:  # local optimum
                    break
                cur = nb
                cur_fit += delta

            if cur_fit < global_best_fit:
                global_best_fit, global_best = cur_fit, cur

        return global_best, float(global_best_fit)

    def assemble_image(self, pieces, order):
        n = self.rows * self.cols
        if len(order) != n or set(order) != set(range(n)):
            order = self._repair_perm(order)
        grid = np.array(order, dtype=int).reshape(self.rows, self.cols)
        w, h = pieces[0].size
        out = Image.new("RGB", (w*self.cols, h*self.rows))
        for r in range(self.rows):
            for c in range(self.cols):
                out.paste(pieces[grid[r, c]], (c*w, r*h))
        return out


# =========================
# RUNNER + Evaluator (auto nhận biết biểu diễn)
# =========================
def invert_perm_c2o_to_o2c(c2o):
    """c2o[c]=o  ->  o2c[o]=c"""
    n = len(c2o)
    o2c = [0]*n
    for c, o in enumerate(c2o):
        o2c[o] = c
    return o2c

def invert_perm_o2c_to_c2o(o2c):
    """o2c[o]=c  ->  c2o[c]=o"""
    n = len(o2c)
    c2o = [0]*n
    for o, c in enumerate(o2c):
        c2o[c] = o
    return c2o

class PuzzleRunnerLocal:
    def __init__(self, solver, pieces_list, image_dir, output_dir, output_img_dir, y_true_csv):
        self.solver = solver
        self.pieces_list = pieces_list
        self.image_dir = image_dir
        self.output_dir = output_dir
        self.output_img_dir = output_img_dir
        self.y_true_csv = y_true_csv
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.output_img_dir, exist_ok=True)

    def run_all(self):
        results = []
        image_files = sorted([f for f in os.listdir(self.image_dir)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

        for idx, pieces in enumerate(tqdm(self.pieces_list, desc="Local search for images")):
            best_order, _ = self.solver.run(pieces)  # c->o (cell->piece)
            image_name = image_files[idx]

            # Lưu ảnh lắp theo c->o
            img = self.solver.assemble_image(pieces, best_order)
            img.save(os.path.join(self.output_img_dir, f"{image_name}_solved.png"))

            # Ghi trực tiếp c->o ra CSV (rất tự nhiên: piece_at_r_c)
            results.append([image_name] + best_order[:])

        # Ghi output.csv (c->o)
        output_csv = os.path.join(self.output_dir, "output.csv")
        with open(output_csv, "w", newline="") as f:
            writer = csv.writer(f)
            header = ["image_filename"] + [
                f"piece_at_{r}_{c}" for r in range(self.solver.rows) for c in range(self.solver.cols)
            ]
            writer.writerow(header)
            writer.writerows(results)

        print(f"Saved output to {output_csv}")
        print(f"Solved images saved to {self.output_img_dir}")
        return output_csv

    def evaluate(self, output_csv):
        df_pred = pd.read_csv(output_csv)
        df_true = pd.read_csv(self.y_true_csv)

        correct_count = 0
        ppa_scores = []

        true_map = {row['image_filename']: row.values[1:].astype(int)
                    for _, row in df_true.iterrows()}

        for _, row in df_pred.iterrows():
            fname = row['image_filename']
            if fname not in true_map:
                continue
            pred = row.values[1:].astype(int)  # chúng ta xuất c->o
            gt   = true_map[fname]            # chưa chắc c->o hay o->c

            # 1) GT là c->o: so trực tiếp
            match_direct = (pred == gt).sum()

            # 2) GT là o->c: đảo về c->o rồi so
            if set(gt) == set(range(len(gt))):
                gt_as_c2o = np.array(invert_perm_o2c_to_c2o(gt), dtype=int)
                match_inv = (pred == gt_as_c2o).sum()
            else:
                match_inv = -1

            match_best = max(match_direct, match_inv)
            ppa_scores.append(match_best / len(gt))
            if match_best == len(gt):
                correct_count += 1

        total = len(df_true)
        acc = (correct_count / total) * 100 if total else 0.0
        mean_ppa = float(np.mean(ppa_scores)) if ppa_scores else 0.0

        print(f"\nTotal images: {total}")
        print(f"Correctly solved: {correct_count}/{total} ({acc:.2f}%)")
        print(f"Average PPA: {mean_ppa:.4f}")


In [3]:
ls_solver = LocalSearchPuzzleSolver(
    rows=ROWS, cols=COLS,
    max_iters=10000,
    restarts=5,
    rng_seed=42
)

runner = PuzzleRunnerLocal(
    solver=ls_solver,
    pieces_list=all_pieces,
    image_dir=image_path,
    output_dir="output_data",
    output_img_dir="output_data/output_images",
    y_true_csv="input_data/Y_test.csv"
)

out_csv = runner.run_all()
runner.evaluate(out_csv)


Local search for images:   0%|          | 0/100 [00:00<?, ?it/s]

Local search for images: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it]

Saved output to output_data\output.csv
Solved images saved to output_data/output_images

Total images: 100
Correctly solved: 72/100 (72.00%)
Average PPA: 0.7547
